### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [1]:
import time
import pandas as pd
import numpy as np
import copy as c

from tqdm import tqdm

ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None,
                         names=['movie_id', 'name', 'category'], engine='python')

implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

EPS = 1e-5
TOY_MOVIE_ID = 1
TEST_USER_ID = 4

def calc_score(rate, n, w, h, shift_w=0, shift_h=0, l=0):
    users = rate.to_numpy()[:, 0] - 1
    movies = rate.to_numpy()[:, 1] - 1
    score = rate.to_numpy()[:, 2]
    return np.linalg.norm(np.array(w @ h + shift_w + shift_h + l)[users, movies] - score) / n


def similars(rate, index, count):
    distances = [[x, np.linalg.norm(np.linalg.norm(old_h[:, x - 1] - old_h[:, index - 1]))]
                 for x in (rate.movie_id.sort_values().unique())]
    distances.sort(key=lambda x: x[1])
    return pd.concat([movie_info.loc[movie_info.movie_id == i] for i in [i[0] for i in distances[:count + 1]]])

recommend = lambda index, qty: pd.concat([movie_info.loc[movie_info.movie_id == i]
               for i in ([i for i in (np.argsort(np.array(s)[index - 1])[::-1] + 1)
                          if i not in set(ratings.loc[ratings.user_id == index].movie_id)][:qty])])

In [2]:
start_time = time.time()


def svg(qty_features, learning_rate, reg_parameter, rmse_rate, qty_it):
    n = ratings.shape[0]
    w = np.random.uniform(0, 1 / np.sqrt(qty_features), (ratings.user_id.max(), qty_features))
    h = np.random.uniform(0, 1 / np.sqrt(qty_features), (qty_features, ratings.movie_id.max()))
    shift_w = np.random.normal(2.5, 0.5, ratings.user_id.max()).reshape(-1, 1)
    shift_h = np.random.normal(2.5, 0.5, ratings.movie_id.max()).reshape(1, -1)
    l = np.mean(ratings.rating.mean())
    score = -1
    iteration = 0
    while iteration < qty_it:
        if iteration % rmse_rate == 0:
            score = calc_score(ratings, n, w, h, shift_w, shift_h, l)
            if score > EPS:
                print("Current iteration %d working time: %s seconds.\n RMSE = %.8f\n" % (
                    iteration, time.time() - start_time, score))
            else:
                break
        i, q, r = ratings.to_numpy()[np.random.randint(n)]
        error = w[i - 1, :] @ h[:, q - 1] + shift_w[i - 1, :] + shift_h[:, q - 1] + l - r
        w[i - 1, :] -= learning_rate * (error * h[:, q - 1].T + reg_parameter * w[i - 1, :])
        h[:, q - 1] -= learning_rate * (error * w[i - 1, :].T + reg_parameter * h[:, q - 1])
        shift_w[i - 1, :] -= learning_rate * (error + reg_parameter * shift_w[i - 1, :])
        shift_h[:, q - 1] -= learning_rate * (error + reg_parameter * shift_h[:, q - 1])
        iteration += 1
    return w @ h + shift_w + shift_h + l, h, score

s, old_h, rmse = svg(64, 1e-2, 1e-5, 2e6, 1e7)
print("Total working time: %s seconds. RMSE = %.8f" % (time.time() - start_time, rmse))

Current iteration 0 working time: 0.44440269470214844 seconds.
 RMSE = 0.00542739

Current iteration 2000000 working time: 61.17354464530945 seconds.
 RMSE = 0.00096322

Current iteration 4000000 working time: 121.78257632255554 seconds.
 RMSE = 0.00088009

Current iteration 6000000 working time: 182.35857772827148 seconds.
 RMSE = 0.00079986

Current iteration 8000000 working time: 242.86551642417908 seconds.
 RMSE = 0.00073553

Total working time: 303.222318649292 seconds. RMSE = 0.00073553


In [3]:
similars(ratings, TOY_MOVIE_ID, 40)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2692,2761,"Iron Giant, The (1999)",Animation|Children's
1838,1907,Mulan (1998),Animation|Children's
2021,2090,"Rescuers, The (1977)",Animation|Children's
1551,1592,Air Bud (1997),Children's|Comedy
358,362,"Jungle Book, The (1994)",Adventure|Children's|Romance
1016,1029,Dumbo (1941),Animation|Children's|Musical
219,222,Circle of Friends (1995),Drama|Romance


In [4]:
recommend(TEST_USER_ID, 40)

,movie_id,name,category
764,774,Wend Kuuni (God's Gift) (1982),Drama
1127,1143,Three Lives and Only One Death (1996),Comedy
1418,1443,"Tickle in the Heart, A (1996)",Documentary
399,403,Two Crimes (1995),Comedy|Crime|Drama
2208,2277,Somewhere in the City (1997),Drama
2478,2547,Harvest (1998),Drama
638,643,Peanuts - Die Bank zahlt alles (1996),Comedy
1093,1109,Charm's Incidents (1996),Drama
1839,1908,Resurrection Man (1998),Drama|Thriller
763,773,Touki Bouki (Journey of the Hyena) (1973),Drama


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [5]:
EPS = 1e-4

start_time = time.time()

def als(qty_features, learning_rate, reg_parameter, rmse_rate, qty_it):
    n = implicit_ratings.shape[0]
    w = np.random.uniform(0, 1 / np.sqrt(qty_features), (implicit_ratings.user_id.max(), qty_features))
    h = np.random.uniform(0, 1 / np.sqrt(qty_features), (qty_features, implicit_ratings.movie_id.max()))
    iteration = 0
    while iteration < qty_it:
        if iteration % rmse_rate == 0:
            score = calc_score(implicit_ratings, n, w, h)
            if score > EPS:
                print("Current iteration %d working time: %s seconds.\n RMSE = %.8f\n" % (
                    iteration, time.time() - start_time, score))
            else:
                break
        error = c.deepcopy(np.array(w @ h))
        error[implicit_ratings.to_numpy()[:, 0] - 1, implicit_ratings.to_numpy()[:, 1] - 1] -= implicit_ratings.to_numpy()[:, 2]
        if iteration % 2 == 1:
            w -= learning_rate * (error @ h.T + reg_parameter * w)
        else:
            h -= learning_rate * (w.T @ error + reg_parameter * h)
        iteration += 1
    return w @ h, h, score

s, old_h, rmse = als(64, 1e-3, 1e-3, 50, 200)
print("Total working time: %s seconds. RMSE = %.8f" % (time.time() - start_time, rmse))

Current iteration 0 working time: 0.42238378524780273 seconds.
 RMSE = 0.00550033

Current iteration 50 working time: 18.56886053085327 seconds.
 RMSE = 0.00395095

Current iteration 100 working time: 36.74936771392822 seconds.
 RMSE = 0.00369796

Current iteration 150 working time: 54.98092579841614 seconds.
 RMSE = 0.00368126

Total working time: 72.88618302345276 seconds. RMSE = 0.00368126


In [6]:
similars(implicit_ratings, TOY_MOVIE_ID, 25)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
360,364,"Lion King, The (1994)",Animation|Children's|Musical
2692,2761,"Iron Giant, The (1999)",Animation|Children's
1838,1907,Mulan (1998),Animation|Children's
2618,2687,Tarzan (1999),Animation|Children's
1526,1566,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
2252,2321,Pleasantville (1998),Comedy


In [7]:
recommend(TEST_USER_ID, 25)

,movie_id,name,category
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1271,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1284,1304,Butch Cassidy and the Sundance Kid (1969),Action|Comedy|Western
957,969,"African Queen, The (1951)",Action|Adventure|Romance|War
1568,1610,"Hunt for Red October, The (1990)",Action|Thriller
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
453,457,"Fugitive, The (1993)",Action|Thriller
1238,1258,"Shining, The (1980)",Horror


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [9]:
start_time = time.time()

def brp(qty_features, learning_rate, reg_parameter, qty_it):
    n = implicit_ratings.shape[0]
    w = np.random.uniform(0, 1 / np.sqrt(qty_features), (implicit_ratings.user_id.max(), qty_features))
    h = np.random.uniform(0, 1 / np.sqrt(qty_features), (qty_features, implicit_ratings.movie_id.max()))
    iteration = 0

    sets = {}
    for i in implicit_ratings.user_id.unique():
        set_positively_reviewed = set(implicit_ratings.loc[implicit_ratings.user_id == i].movie_id)
        set_negatively_reviewed = set(implicit_ratings.movie_id.unique()) - set_positively_reviewed
        sets[i] = MyPair(list(set_positively_reviewed), list(set_negatively_reviewed))

    while iteration < qty_it:
        score = calc_score(implicit_ratings, n, w, h)
        if score <= EPS:
            break
        for i in tqdm(implicit_ratings.user_id.unique()):
            for i_pos, i_neg in sets[i].generate():
                error = 1.0 / (1 + np.exp(w[i - 1, :] @ h[:, i_pos - 1] - w[i - 1, :] @ h[:, i_neg - 1]))
                w[i - 1, :] += learning_rate * (error * (h[:, i_pos - 1].T - h[:, i_neg - 1].T) - reg_parameter * w[i - 1, :])
                h[:, i_pos - 1] += learning_rate * (error * w[i - 1, :].T - reg_parameter * h[:, i_pos - 1])
                h[:, i_neg - 1] += learning_rate * (error * -w[i - 1, :].T - reg_parameter * h[:, i_neg - 1])
        print("Current iteration %d working time: %s seconds.\n Score = %.12f\n" % (
            iteration, time.time() - start_time, score))
        iteration += 1
    return (w @ h), h, score
        
class MyPair:
    def __init__(self, positive, negative):
        self.positively_received = positive
        self.negatively_received = negative

    def generate(self):
        for positive in self.positively_received:
            for negative in np.random.choice(self.negatively_received, 7):
                yield positive, negative


s, old_h, score = brp(64, 1e-2, 1e-4, 6)
print("Total working time: %s seconds. %.12f" % (time.time() - start_time, score))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6038/6038 [02:52<00:00, 34.96it/s]


Current iteration 0 working time: 210.45108890533447 seconds.
 Score = 0.005501003630



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6038/6038 [02:57<00:00, 34.07it/s]


Current iteration 1 working time: 388.1023950576782 seconds.
 Score = 0.003025909793



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6038/6038 [02:49<00:00, 35.62it/s]


...

Current iteration 3 working time: 725.4006593227386 seconds.
 Score = 0.002393228557



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6038/6038 [02:47<00:00, 35.98it/s]


Current iteration 4 working time: 893.6194109916687 seconds.
 Score = 0.002266759528



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6038/6038 [02:47<00:00, 35.99it/s]


Current iteration 5 working time: 1061.803126335144 seconds.
 Score = 0.002241164519

Total working time: 1062.3266017436981 seconds. 0.002241164519


In [10]:
similars(implicit_ratings, TOY_MOVIE_ID, 25)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
1179,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
1245,1265,Groundhog Day (1993),Comedy|Romance
2647,2716,Ghostbusters (1984),Comedy|Horror
2918,2987,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
584,588,Aladdin (1992),Animation|Children's|Comedy|Musical
33,34,Babe (1995),Children's|Comedy|Drama
1239,1259,Stand by Me (1986),Adventure|Comedy|Drama


In [11]:
recommend(TEST_USER_ID, 25)

,movie_id,name,category
847,858,"Godfather, The (1972)",Action|Crime|Drama
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
2789,2858,American Beauty (1999),Comedy|Drama
1575,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
604,608,Fargo (1996),Crime|Drama|Thriller
2693,2762,"Sixth Sense, The (1999)",Thriller
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama


### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных